In [1]:
from scipy.linalg import qr, pinv, solve, norm
from numpy.random import randn
import numpy as np

In [2]:
# generate random data matrix
n,d = 6,4
X = randn(n,d)

# optional: give it linearly dependent columns
# X[:,3] = X[:,2]

# Understanding the pseudoinverse

In [3]:
# form pseudoinverse
Xd = pinv(X)

In [4]:
# X†X ≈ I_d
Xd @ X

array([[ 1.00000000e+00,  6.93889390e-17, -2.77555756e-16,
         2.91433544e-16],
       [-2.22044605e-16,  1.00000000e+00,  7.11236625e-17,
         0.00000000e+00],
       [ 3.67761377e-16, -1.66533454e-16,  1.00000000e+00,
        -1.08246745e-15],
       [ 1.11022302e-16,  2.03613168e-16, -1.11022302e-16,
         1.00000000e+00]])

In [5]:
np.allclose(Xd @ X, np.identity(4))

True

In [6]:
# XX† !≈ I_n
X @ Xd

array([[ 0.59490125, -0.00762251, -0.28842487, -0.19323077,  0.24401744,
        -0.24670651],
       [-0.00762251,  0.27406379,  0.08921053,  0.16569656,  0.37790724,
         0.14376014],
       [-0.28842487,  0.08921053,  0.78230537, -0.15965734,  0.12505972,
        -0.19500221],
       [-0.19323077,  0.16569656, -0.15965734,  0.86832311,  0.02929838,
        -0.15230148],
       [ 0.24401744,  0.37790724,  0.12505972,  0.02929838,  0.66099527,
         0.07227588],
       [-0.24670651,  0.14376014, -0.19500221, -0.15230148,  0.07227588,
         0.8194112 ]])

In [7]:
np.allclose(X @ Xd, np.identity(6))

False

In [8]:
Q,R = qr(X)
Q,R = qr(X, mode='economic')

In [9]:
np.allclose(X, Q @ R)

True

In [10]:
Q

array([[-0.23776564,  0.30194971, -0.38065815, -0.54981314],
       [-0.19147557, -0.45966854,  0.00807992, -0.16137054],
       [ 0.29740618, -0.48970728, -0.48785745,  0.46479761],
       [ 0.45324307, -0.38303946,  0.56382848, -0.44527749],
       [-0.38146712, -0.53918846, -0.29853432, -0.36828138],
       [-0.68367223, -0.14238878,  0.45826183,  0.34889112]])

In [11]:
R

array([[ 4.11845228, -0.82070139,  1.26471698, -0.16018948],
       [ 0.        , -1.73776746, -1.27595435,  1.63501515],
       [ 0.        ,  0.        , -1.80247779, -0.21317942],
       [ 0.        ,  0.        ,  0.        , -2.09548679]])

In [12]:
print(np.allclose(Q.T @ Q, np.identity(Q.shape[1])))
Q.T @ Q

True


array([[ 1.00000000e+00,  9.23117593e-17, -1.81805482e-17,
        -3.41840997e-17],
       [ 9.23117593e-17,  1.00000000e+00, -7.94276235e-18,
         1.05564894e-16],
       [-1.81805482e-17, -7.94276235e-18,  1.00000000e+00,
        -1.77225919e-17],
       [-3.41840997e-17,  1.05564894e-16, -1.77225919e-17,
         1.00000000e+00]])

In [13]:
# form data from noisy linear model
wtrue = randn(d)
y = X.dot(wtrue) + .1*randn(n)

In [14]:
# solve least squares problem to estimate w
w = solve(R, Q.T @ y)

In [15]:
# how good is our estimate?
norm(w - wtrue)

0.046586758785048095

In [16]:
# compute mean square error
def mse(y,z):
    return sum((y-z)**2)/len(y)
    
mse(y,X.dot(w))

0.005055452124355269

In [ ]:
# in other languages, we can use the shorthand
w_backslash = X \ y
norm(w_backslash - w)